In [1]:
import pandas as pd


In [ ]:
df = pd.read_csv('fake_news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [7]:
df = df.dropna()

In [8]:
X = df.drop('label', axis=1)
y = df['label']

In [23]:
messages = df.copy()
messages.reset_index(inplace=True)

In [24]:
X.shape, y.shape

((18285, 4), (18285,))

In [25]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [PorterStemmer().stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [30]:
voc_size = 10000
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[7292, 1718, 9255, 2727, 3505, 5186, 568, 6718, 6882, 3332],
 [6119, 2011, 3013, 2461, 8564, 9563, 5059],
 [4643, 2588, 2216, 6282],
 [6601, 559, 5549, 2615, 2095, 8618],
 [2762, 8564, 7711, 4862, 6774, 9376, 8564, 8646, 3930, 3510],
 [7351,
  7905,
  635,
  3114,
  8262,
  6504,
  9099,
  8212,
  9458,
  6747,
  4212,
  6304,
  4273,
  1159,
  5059],
 [113, 1583, 7110, 4325, 577, 4877, 2357, 2560, 8069, 9913, 6426],
 [9454, 59, 4820, 1578, 9883, 8058, 6504, 3102, 8069, 9913, 6426],
 [4805, 9594, 9386, 6169, 55, 111, 9561, 6722, 6504, 9890],
 [5345, 5583, 6387, 2164, 3847, 6438, 3301, 5353],
 [5031, 899, 941, 1391, 247, 7706, 4698, 8176, 8637, 4652, 6280],
 [2615, 4265, 3505, 111, 6504, 9883],
 [9568, 4871, 7019, 9118, 5973, 9085, 2819, 2976, 4236],
 [9260, 2505, 3407, 2262, 2346, 3598, 384, 8069, 9913, 6426],
 [5020, 2684, 1206, 8705, 1447, 8069, 9913, 6426],
 [5766, 7953, 5428, 6429, 25, 7796, 1178, 7843, 3542, 3357],
 [6721, 192, 2011],
 [2272, 8482, 1393, 5309, 6504, 8154, 3216, 5

In [31]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 6718 6882 3332]
 [   0    0    0 ... 8564 9563 5059]
 [   0    0    0 ... 2588 2216 6282]
 ...
 [   0    0    0 ... 8069 9913 6426]
 [   0    0    0 ... 6163 5886 4391]
 [   0    0    0 ... 9800  516 3220]]


In [32]:
 model = Sequential()
 model.add(Embedding(voc_size, 40, input_length=sent_length))
 model.add(Bidirectional(LSTM(400)))
 model.add(Dense(1, activation='sigmoid'))
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [33]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [35]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.8125 - loss: 0.3752 - val_accuracy: 0.9178 - val_loss: 0.1996
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9526 - loss: 0.1197 - val_accuracy: 0.9195 - val_loss: 0.1874
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9797 - loss: 0.0612 - val_accuracy: 0.9100 - val_loss: 0.2514
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9843 - loss: 0.0449 - val_accuracy: 0.9100 - val_loss: 0.2749
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9934 - loss: 0.0235 - val_accuracy: 0.9112 - val_loss: 0.3864
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9951 - loss: 0.0205 - val_accuracy: 0.9127 - val_loss: 0.3618
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9976 - loss: 0.0109 - val_accuracy: 0.9082 - val_loss: 0.3676
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9979 - loss: 0.0082 - val_acc

In [36]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >=0.5,1,0)

from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test, y_pred)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [37]:
ac

0.9065451532725767

In [38]:
cm

array([[3118,  301],
       [ 263, 2353]])

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

